<a id="primero"></a>
## 1. Predicción de Entalpía de Atomización


Las simulaciones de propiedades moleculares son computacionalmente costosas y requieren de un arduo trabajo científico. El objetivo de esta sección corresponde a la utilización de métodos de aprendizaje automático supervisado (Redes Neuronales Artificiales) para predecir propiedades moleculares, en este caso la Energía de Atomización o Entalpía de Atomización, a partir de una base de datos de simulaciones obtenida mediante __[Quantum Espresso](http://www.quantum-espresso.org/)__. Si esto se lograse hacer con gran precisión, se abrirían muchas posibilidades en el diseño computacional y el descubrimiento de nuevas moléculas, compuestos y fármacos.

<img src="https://pubs.rsc.org/services/images/RSCpubs.ePlatform.Service.FreeContent.ImageService.svc/ImageService/Articleimage/2012/NR/c2nr11543c/c2nr11543c-f4.gif" title="Title text" width="40%"/>


La **entalpía de atomización** es la cantidad de variación de entalpía cuando los enlaces de un compuesto se rompen y los componentes se reducen a átomos individuales. Tal como se ha indicado, su tarea es la de predecir dicho nivel a partir de los atributos enunciados en el dataset puesto a vuestra disposición en *moodle*.

**Nota: Debido a lo mucho que toma realizar los experimentos, los errores se guardan en archivos caché `.npy`, estos deben borrarse si se quiere repetir los experimentos.**

**SI NO SE VISUALIZAN LOS WIDGETS, CORRA LAS CELDAS DE NUEVO, NO SE ENTRENARÁN REDES**.

In [1]:
import os

import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold

import keras
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.layers import Dropout
from keras.optimizers import SGD, Adam, RMSprop, Adagrad, Adadelta
from keras.regularizers import l1,l2

import matplotlib.pyplot as plt

from ipywidgets import interact

Using TensorFlow backend.


#### **NOTA:**
Debido a lo mucho que toma realizar los experimentos, los errores se guardan en archivos caché `.npy`, estos deben borrarse si se quiere repetir los experimentos.

---
a) Construya un *dataframe* con los datos a analizar y descríbalo brevemete. Además, realice la división de éste en los conjuntos de entrenamiento, validación y testeo correspondientes. Comente por qué se deben eliminar ciertas columnas.

In [3]:
datos = pd.read_csv("roboBohr.csv")
print("datos.shape:",datos.shape)
datos.info()
datos.describe()

datos.shape: (16242, 1278)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16242 entries, 0 to 16241
Columns: 1278 entries, Unnamed: 0 to Eat
dtypes: float64(1276), int64(2)
memory usage: 158.4 MB


,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,1267,1268,1269,1270,1271,1272,1273,1274,pubchem_id,Eat
count,16242.000000,16242.000000,16242.000000,16242.000000,16242.000000,16242.000000,16242.000000,16242.000000,16242.000000,16242.000000,...,16242.000000,16242.000000,16242.000000,16242.000000,16242.000000,16242.000000,16242.000000,16242.000000,16242.000000,16242.000000
mean,8139.041805,115.715266,22.445723,20.474191,18.529573,17.169350,15.816888,15.133152,14.471534,13.960759,...,0.000134,0.000133,0.003879,0.000131,0.000129,0.002155,0.000127,0.001201,33107.484300,-11.178969
std,4698.182820,113.198503,8.659586,7.670481,6.485777,5.512560,4.179691,3.885091,3.503075,3.357136,...,0.002728,0.002705,0.043869,0.002676,0.002633,0.032755,0.002594,0.024472,23456.785147,3.659133
min,0.000000,36.858105,2.906146,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,-23.245373
25%,4068.250000,73.516695,17.969345,16.228071,15.165862,13.744092,13.653146,13.637784,12.759519,12.587359,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,12298.250000,-13.475805
50%,8142.500000,73.516695,20.662511,18.631287,17.690729,16.020040,15.156646,13.848274,13.659233,13.652832,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,27731.500000,-10.835211
75%,12207.750000,73.516695,21.132432,20.739496,18.712895,18.297501,17.639688,16.154918,15.499474,14.900585,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,55020.750000,-8.623903
max,16272.000000,388.023441,73.563510,66.269180,66.268891,66.268756,66.268196,66.264158,66.258487,66.258177,...,0.062225,0.061999,0.500000,0.061534,0.059760,0.500000,0.057834,0.500000,74980.000000,-0.789513


In [4]:
datos.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,1267,1268,1269,1270,1271,1272,1273,1274,pubchem_id,Eat
0,0,73.516695,17.817765,12.469551,12.458130,12.454607,12.447345,12.433065,12.426926,12.387474,...,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,25004,-19.013763
1,1,73.516695,20.649126,18.527789,17.891535,17.887995,17.871731,17.852586,17.729842,15.864270,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25005,-10.161019
2,2,73.516695,17.830377,12.512263,12.404775,12.394493,12.391564,12.324461,12.238106,10.423249,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25006,-9.376619
3,3,73.516695,17.875810,17.871259,17.862402,17.850920,17.850440,12.558105,12.557645,12.517583,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25009,-13.776438
4,4,73.516695,17.883818,17.868256,17.864221,17.818540,12.508657,12.490519,12.450098,10.597068,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25011,-8.537140


In [5]:
datos.drop(columns=['Unnamed: 0','pubchem_id'],axis=1,inplace=True)
total = len(datos)
df_trai = datos[:int(0.6*total)]                       #60% de los datos
df_vali = datos[int(0.6*total):int(0.85*total)]        #25% de los datos
df_test = datos[int(0.85*total)::]                     #15% restante

La columna `Unnamed: 0` representa el id del compuesto dentro del dataset, y `pubchem_id` parece ser un id general para identificar el compuesto. Ambas columnas se remueven porque la asignación de estos índices es arbitraria y su valor no debería estar relacionado con el resultado que debe entregar nuestro modelo (la Entalpía de Atomización, correspondiente a la columna `Eat`). Aunque el modelo debería detectar que no hay correlación entre `Eat` y estos atributos, es mejor removerlos para no *confundir* el aprendizaje.

---
a.1) Una buena práctica es la de normalizar los datos antes de trabajar con el modelo. **Explique por qué se aconseja dicho preprocesamiento**

In [6]:
# Get scaler and scale data
scaler = StandardScaler().fit(df_trai)
X_trai_scaled = pd.DataFrame(scaler.transform(df_trai),columns=df_trai.columns)
X_vali_scaled = pd.DataFrame(scaler.transform(df_vali),columns=df_vali.columns)
X_test_scaled = pd.DataFrame(scaler.transform(df_test),columns=df_test.columns)
# Get targets
y_trai = df_trai.pop('Eat').values.reshape(-1,1)
y_vali = df_vali.pop('Eat').values.reshape(-1,1)
y_test = df_test.pop('Eat').values.reshape(-1,1)
# Remove targets from attributes
X_trai_scaled.drop(columns=['Eat'],axis=1,inplace=True)
X_vali_scaled.drop(columns=['Eat'],axis=1,inplace=True)
X_test_scaled.drop(columns=['Eat'],axis=1,inplace=True)
# 
print("X_trai_scaled",X_trai_scaled.shape)
print("X_vali_scaled",X_vali_scaled.shape)
print("X_test_scaled",X_test_scaled.shape)

X_trai_scaled (9745, 1275)
X_vali_scaled (4060, 1275)
X_test_scaled (2437, 1275)


Muchos modelos de aprendizaje son suceptibles a la escala de los atributos, por ese motivo, atributos con órdenes de magnitud mayores pueden afectar más a los mismos.
Para hacer el aprendizaje **independiente** de las unidades de medición en que se presentan estos atributos y evitar un **bias** del aprendizaje hacia unos por sobre otros, es que se normalizan.

---
b) Muestre en un gráfico el error cuadrático (MSE) para el conjunto de entrenamiento y de pruebas vs número de *epochs* de entrenamiento, para una red *feedforward* de 3 capas, con 256 unidades ocultas y función de activación sigmoidal. Entrene la red usando gradiente descendente estocástico con tasa de aprendizaje (learning rate) 0.01 y 250 epochs de entrenamiento, en el conjunto de entrenamiento y de validación. Comente. Si observara divergencia durante el entrenamiento, determine si esto ocurre para cada repetición del experimento.

In [6]:
# Create model:
def create_model(activation='sigmoid',initializer='uniform',
                 lr=None,decay=None,optimizer=None,
                 layer1_reg=None,layer2_reg=None,dropout=None,
                 neurons=256):
    # Define model
    model = Sequential()
    model.add(Dense(neurons,input_dim=X_trai_scaled.shape[1],
                    kernel_initializer=initializer,activation=activation,
                    kernel_regularizer=layer1_reg))
    if dropout:
        model.add(Dropout(dropout))
    model.add(Dense(1, kernel_initializer=initializer,activation="linear",
                    kernel_regularizer=layer2_reg))
    # Set optimizer
    if optimizer is None:
        # default values
        if lr is None:
            if activation=="sigmoid": lr = 0.01
            elif activation=="relu" : lr = 0.001
        if decay is None: decay = 0.0
        optimizer = SGD(lr=lr,decay=decay)
    else:
        assert lr is None and decay is None
    # Compile model with optimizer
    model.compile(optimizer=optimizer,loss='mean_squared_error')
    return model

In [7]:
def gradual_color(i,total):
    if i<total//2:
        return (2.0*i/float(total),0.0,0.0)
    else:
        return (1.0,2.0*(i-total//2)/float(total),0.0)

class PlotErrors(keras.callbacks.Callback):
    
    def __init__(self,text=None,labels=None,gradual=False,save_name=None):
        self.trai_err = []
        self.vali_err = []
        self.save_name = save_name
        self.loaded = False
        # Load cache
        if os.path.isfile(self.save_name):
            print("Loaded from %s"%save_name)
            data = np.load(self.save_name)
            print("data.shape",data.shape)
            self.trai_err = data[0]
            self.vali_err = data[1]
            self.loaded = True
        # Labels
        self.labels = labels
        self.gradual = gradual
        self.trai_title = "Training error v/s epoch"
        self.vali_title = "Validaton error v/s epoch"
        if text:
            self.trai_title = self.trai_title+" for different "+text
            self.vali_title = self.vali_title+" for different "+text
        self.widget = interact(self.display).widget

    def display(self):
        fig, ax = plt.subplots(1,2,figsize=(12,6),sharex=True,sharey=True)
        ax[0].set_ylim(0.01,1000)
        # Legend and colors:
        legc = {}
        # Train error:
        ax[0].set(xlabel='epoch',ylabel='training error',title=self.trai_title)
        for i in range(len(self.trai_err)):
            #
            if self.labels and self.gradual: legc['c'] = gradual_color(i,len(self.labels))
            if self.labels: legc['label'] = self.labels[i]
            #
            x = np.arange(1,1+len(self.trai_err[i]))
            ax[0].semilogy(x,self.trai_err[i],linewidth=2,**legc)
        ax[0].grid()
        if self.labels: ax[0].legend()
        # Validation error
        ax[1].set(xlabel='epoch',ylabel='validation error',title=self.vali_title)
        for i in range(len(self.vali_err)):
            #
            if self.labels and self.gradual: legc['c'] = gradual_color(i,len(self.labels))
            if self.labels: legc['label'] = self.labels[i]
            #
            x = np.arange(1,1+len(self.vali_err[i]))
            ax[1].semilogy(x,self.vali_err[i],linewidth=2,**legc)
        ax[1].grid()
        if self.labels: ax[1].legend()
        #
        plt.show()

    def on_batch_end(self, batch, logs={}):
        # Stop if loss is NaN
        loss = logs.get('loss')
        if np.isnan(loss):
            self.model.stop_training = True
            print("Loss is nan! Stop.")

    def on_train_begin(self, logs={}):
        self.trai_err.append([])
        self.vali_err.append([])
    
    def on_train_end(self,logs={}):
        if self.save_name and (self.labels is None or len(self.vali_err)==len(self.labels)):
            siz_x = max([len(x) for x in self.trai_err])
            u = np.zeros((2,len(self.trai_err),siz_x))
            u[:,:] = np.nan
            for i in range(len(self.trai_err)):
                u[0,i,:len(self.trai_err[i])] = self.trai_err[i]
                u[1,i,:len(self.vali_err[i])] = self.vali_err[i]
            np.save(self.save_name,u)

    def on_epoch_end(self, epoch, logs={}):
        vali_loss = logs.get('val_loss')
        trai_loss = logs.get('loss')
        self.vali_err[-1].append(vali_loss)
        self.trai_err[-1].append(trai_loss)
        self.widget.update()
        return

In [8]:
EPOCHS = 250
EXPERIMENTS = 5
BATCH_SIZE = 64

In [9]:
callback = PlotErrors("experiments",save_name="1b.npy")
if not callback.loaded:
    for i in range(EXPERIMENTS):
        model = create_model(activation="sigmoid")
        model.fit(X_trai_scaled, y_trai, epochs=EPOCHS, verbose=0, batch_size=BATCH_SIZE,
            validation_data=(X_vali_scaled, y_vali), callbacks=[callback])
        del model

Loaded from 1b.npy
data.shape (2, 5, 250)


interactive(children=(Output(),), _dom_classes=('widget-interact',))

Se puede notar como ambos errores disminuyen cada vez más lentamente, este comportamiento no difiere casi nada para la misma red inicializada con pesos diferentes. 

El error de testing no empeora pese a las 250 epoch, lo que sugiere que se puede entrenar más para obtener una mejor accuracy. 

Se puede ver que el error de validación varía bastante, lo que se puede deber a que el dataset es pequeño y que el conjunto de validación no participa en el entrenamiento.

---
c) Repita el paso anterior, utilizado ’**ReLU**’ como función de activación y compare con lo obtenido en b).

In [10]:
callback = PlotErrors("experiments",save_name="1c.npy")
if not callback.loaded:
    for i in range(EXPERIMENTS):
        initi = keras.initializers.RandomUniform(-1e-5,1e-5)
        model = create_model(activation="relu",initializer=initi)
        model.fit(X_trai_scaled, y_trai, epochs=EPOCHS, verbose=0, batch_size=BATCH_SIZE,
            validation_data=(X_vali_scaled, y_vali), callbacks=[callback])
        del model

Loaded from 1c.npy
data.shape (2, 5, 250)


interactive(children=(Output(),), _dom_classes=('widget-interact',))

La convergencia es más **lenta** que con la activación **sigmoide**, además, se puede ver que se está generalizando mal, ya que el error de validación aumenta y es al menos un órden de magnitud mayor.

No se nota **divergencia** en el error de entrenamiento, sin embargo, hay una diferencia significativa en en el **error de validación** para cada uno de los casos de prueba.

Cabe destacar que fue necesario bajar la **tasa de aprendizaje** a 0.001 y utilizar una **inicialización lineal** con **valores pequeños** ya que de otra manera resultaban valores `NaN`.

---
d) Repita b) y c) variando la tasa de aprendizaje (*learning rate*) en un rango sensible. Comente. Si observara divergencia durante el entrenamiento, determine si esto ocurre para cada repetición del experimento.

In [11]:
n_lr = 10
learn_rate = {"sigmoid":np.arange(1,n_lr+1)/(20.0*n_lr),"relu":np.arange(1,n_lr+1)/(200.0*n_lr)}
print(learn_rate["sigmoid"])
print(learn_rate["relu"])

[0.005 0.01  0.015 0.02  0.025 0.03  0.035 0.04  0.045 0.05 ]
[0.0005 0.001  0.0015 0.002  0.0025 0.003  0.0035 0.004  0.0045 0.005 ]


In [12]:
for activ in ("sigmoid","relu"):
    callback = PlotErrors("learning rates",[str(x) for x in learn_rate[activ]],gradual=True,
                          save_name="1d_%s.npy"%activ)
    if not callback.loaded:
        for lr in learn_rate[activ]:
            initi = keras.initializers.RandomUniform(-1e-5,1e-5,seed=42)
            model = create_model(activation=activ,initializer=initi,lr=lr)
            model.fit(X_trai_scaled, y_trai, epochs=EPOCHS, verbose=0, batch_size=BATCH_SIZE,
                validation_data=(X_vali_scaled, y_vali), callbacks=[callback])
            del model

Loaded from 1d_sigmoid.npy
data.shape (2, 10, 250)


interactive(children=(Output(),), _dom_classes=('widget-interact',))

Loaded from 1d_relu.npy
data.shape (2, 10, 250)


interactive(children=(Output(),), _dom_classes=('widget-interact',))

Nótese que para **LR** grandes la pérdida se transformó en `NaN`, lo que terminó el entrenamiento.

Para la activación **sigmoide** se puede notar que la **convergencia** es más rápida mientras **mayor** es la **tasa de aprendizaje**. Para **relu**, con **LR** de 0.003, se logró generalizar mejor, posiblemente porque se pudieron **evitar óptimos locales**.

---
e) Entrene los modelos considerados en b) y c) usando *progressive decay*. Compare y comente.

In [13]:
INIT_LR = 0.02
n_decays = 10
decays = np.logspace(-6,0,n_decays)
print(decays)

[1.00000000e-06 4.64158883e-06 2.15443469e-05 1.00000000e-04
 4.64158883e-04 2.15443469e-03 1.00000000e-02 4.64158883e-02
 2.15443469e-01 1.00000000e+00]


In [14]:
for activ in ("sigmoid","relu"):
    callback = PlotErrors("learning decays",[str(x) for x in decays],gradual=True,
                         save_name="1e_%s.npy"%activ)
    if not callback.loaded:
        for dec in decays:
            initi = keras.initializers.RandomUniform(-1e-5,1e-5,seed=42)
            model = create_model(activation=activ,initializer=initi,lr=INIT_LR,decay=dec)
            model.fit(X_trai_scaled, y_trai, epochs=EPOCHS, verbose=0, batch_size=BATCH_SIZE,
                validation_data=(X_vali_scaled, y_vali), callbacks=[callback])
            del model

Loaded from 1e_sigmoid.npy
data.shape (2, 10, 250)


interactive(children=(Output(),), _dom_classes=('widget-interact',))

Loaded from 1e_relu.npy
data.shape (2, 10, 250)


interactive(children=(Output(),), _dom_classes=('widget-interact',))

Se puede ver que decays muy altos estancan el aprendizaje en cierto punto, si son suficientes permiten a la red converger más rápido y generalizar mejor (ver **sigmoide 0.0001**), sin embargo, si son muy pequeños no tienen un efecto significativo (**sigmoide 2.1544e-05**).

Se colocó una learning rate relativamente alta pero no lo suficiente como para que resulten 
pérdidas `nan`.

<!-- De acuerdo con [esta respuesta](https://stats.stackexchange.com/a/211340), la tasa de aprendizaje dado un *decay* $D$ es:
$$
L_t = L_0 \, \frac{1}{1 +  t \cdot D}
$$ -->

---
f) Entrene los modelos considerados en b) y c) utilizando SGD en mini-*batches*. Experimente con diferentes tamaños del *batch*. Comente.

In [17]:
n_batches = 9
batch_sizes = np.array(np.logspace(11,3,n_batches,base=2),dtype='int')
print(batch_sizes)

[2048 1024  512  256  128   64   32   16    8]


In [18]:
for activ in ("sigmoid","relu"):
    callback = PlotErrors("batch sizes",[str(x) for x in batch_sizes],gradual=True,
                         save_name="1f_%s.npy"%activ)
    if not callback.loaded:
        for bs in batch_sizes:
            initi = keras.initializers.RandomUniform(-1e-5,1e-5,seed=42)
            model = create_model(activation=activ,initializer=initi)
            model.fit(X_trai_scaled, y_trai, epochs=EPOCHS, verbose=0,
                validation_data=(X_vali_scaled, y_vali), callbacks=[callback],
                batch_size=bs)
            del model

Loaded from 1f_sigmoid.npy
data.shape (2, 9, 250)


interactive(children=(Output(),), _dom_classes=('widget-interact',))

Loaded from 1f_relu.npy
data.shape (2, 9, 250)


interactive(children=(Output(),), _dom_classes=('widget-interact',))

Se puede ver que las redes con activación **sigmoide** siguen **generalizando mejor**, mientras más pequeños, los **batch sizes** dan mejores resultados, aunque demoraron más en entrenar. 

---
g) Entrene los modelos obtenidos en b) y c) utilizando estrategias modernas para adaptar la tasa de aprendizaje. Compare los desempeños de adagrad, adadelta, RMSprop y adam. ¿Se observa en algún caso un mejor resultado final? ¿Se observa en algún caso una mayor velocidad de convergencia sobre el dataset de entrenamiento? ¿Sobre el dataset de validación?

In [19]:
optimizers = [SGD,Adam,RMSprop,Adagrad,Adadelta]

In [20]:
for activ in ("sigmoid","relu"):
    callback = PlotErrors("optimizers",[x.__name__ for x in optimizers],
                         save_name="1g_%s.npy"%activ)
    if not callback.loaded:
        for opt in optimizers:
            initi = keras.initializers.RandomUniform(-1e-5,1e-5,seed=42)
            model = create_model(activation=activ,initializer=initi,optimizer=opt(lr=0.01))
            model.fit(X_trai_scaled, y_trai, epochs=EPOCHS, verbose=0, batch_size=BATCH_SIZE,
                    validation_data=(X_vali_scaled, y_vali), callbacks=[callback])
            del model

Loaded from 1g_sigmoid.npy
data.shape (2, 5, 250)


interactive(children=(Output(),), _dom_classes=('widget-interact',))

Loaded from 1g_relu.npy
data.shape (2, 5, 250)


interactive(children=(Output(),), _dom_classes=('widget-interact',))

Para la activación **sigmoide** SGD obtuvo mejores resultados, en cambio para **relu** el mejor fue obtenido con **Adagrad**. **Adagrad** parece confiable para ambos casos.

**RMSprop** y **Adam** varían bastante en el score de validación, en el caso de **relu** también lo hacen en el de entrenamiento y no son efectivas.

---
h) Entrene los modelos obtenidos en b) y c) utilizando regularizadores $l_1$ y $l_2$ (*weight decay*). Compare los desempeños de prueba obtenidos antes y después de regularizar. Experimente con distintos valores del parámetro de regularización y comente. Además evalúe el efecto de regularizar solo la primera capa *vs* la segunda, comente.

In [21]:
regs = [l1(0.001),l1(0.01),l1(0.1),l2(0.001),l2(0.01),l2(0.1)]

In [22]:
for activ in ("sigmoid","relu"):
    callback = PlotErrors("regularizers",["l1=%.3f,l2=%.3f "%(x.l1,x.l2) for x in regs],
                         save_name="1h1_%s.npy"%activ)
    if not callback.loaded:
        for reg in regs:
            initi = keras.initializers.RandomUniform(-1e-5,1e-5,seed=42)
            model = create_model(activation=activ,initializer=initi,
                                 layer1_reg=reg,layer2_reg=reg)
            model.fit(X_trai_scaled, y_trai, epochs=EPOCHS, verbose=0, batch_size=BATCH_SIZE,
                    validation_data=(X_vali_scaled, y_vali), callbacks=[callback])
            del model

Loaded from 1h1_sigmoid.npy
data.shape (2, 6, 250)


interactive(children=(Output(),), _dom_classes=('widget-interact',))

Loaded from 1h1_relu.npy
data.shape (2, 6, 250)


interactive(children=(Output(),), _dom_classes=('widget-interact',))

Los **regularizadores** $l1=0.1$ y $l1=0.01$ son muy **altos** para ambos casos. Mejores resultados se obtienen con las regularizaciones más **bajas**, lo que hace pensar que se debió probar con valores aun más bajos, ya que están afectando mucho la **pérdida**.

In [23]:
dual_regs = {"l1_1layer":(l1(0.01),None),
             "l1_2layer":(None,l1(0.01)),
             "l1_both"  :(l1(0.01),l1(0.01)),
             "l2_1layer":(l2(0.01),None),
             "l2_2layer":(None,l2(0.01)),
             "l2_both"  :(l2(0.01),l2(0.01))}

In [24]:
names = list(dual_regs.keys())
for activ in ("sigmoid","relu"):
    callback = PlotErrors("regularizers",names,
                         save_name="1h2_%s.npy"%activ)
    if not callback.loaded:
        for nam in names:
            initi = keras.initializers.RandomUniform(-1e-5,1e-5,seed=42)
            model = create_model(activation=activ,initializer=initi,
                                 layer1_reg=dual_regs[nam][0],layer2_reg=dual_regs[nam][1])
            model.fit(X_trai_scaled, y_trai, epochs=EPOCHS, verbose=0, batch_size=BATCH_SIZE,
                    validation_data=(X_vali_scaled, y_vali), callbacks=[callback])
            del model

Loaded from 1h2_sigmoid.npy
data.shape (2, 6, 250)


interactive(children=(Output(),), _dom_classes=('widget-interact',))

Loaded from 1h2_relu.npy
data.shape (2, 6, 250)


interactive(children=(Output(),), _dom_classes=('widget-interact',))

Mejores resultados se obtienen con regularizaciones **l2**, en ambos casos la activación **sigmoide** funciona mejor cuando se aplica en la **segunda capa**. En el caso de **relu**, cuando se aplica en ambas mejora más, lo que tiene sentido puesto que en la primera pueden surgir pesos grandes. 

Al menos en este ejemplo las **regularizaciones** no parecen afectar la **capacidad de generalización**.

---
i) Entrene los modelos obtenidos en b) y c) utilizando *Dropout*. Compare los desempeños de prueba obtenidos antes y después de regularizar. Experimente con distintos valores del parámetro de regularización y comente.

In [25]:
dropouts = [0.2,0.4,0.6,0.8]

In [26]:
for activ in ("sigmoid","relu"):
    callback = PlotErrors("dropouts",[str(x) for x in dropouts],
                         save_name="1i_%s.npy"%activ)
    if not callback.loaded:
        for drop in dropouts:
            initi = keras.initializers.RandomUniform(-1e-5,1e-5,seed=42)
            model = create_model(activation=activ,initializer=initi,dropout=drop)
            model.fit(X_trai_scaled, y_trai, epochs=EPOCHS, verbose=0, batch_size=BATCH_SIZE,
                        validation_data=(X_vali_scaled, y_vali), callbacks=[callback])
            del model

Loaded from 1i_sigmoid.npy
data.shape (2, 4, 250)


interactive(children=(Output(),), _dom_classes=('widget-interact',))

Loaded from 1i_relu.npy
data.shape (2, 4, 250)


interactive(children=(Output(),), _dom_classes=('widget-interact',))

Un **dropout** muy **alto** genera pérdidas `NaN`. Mayores dropouts son efectivos para mejorar la capacidad de generalización en las **relu**, sin embargo, son malas en la **sigmoide**, que está obteniendo mejores resultados.

Se pueden ver los efectos del **dropout** en el error de entrenamiento, y esto se puede usar para medir cuando se está eligiendo un **dropout**, muy alto.

---
j) Fijando todos los demás hiper-parámetros del modelo definido en b) y en c), utilice validación cruzada con un número de *folds* igual a *K* = 5 y *K*=10 para determinar el mejor valor correspondiente a un parámetro que usted elija (tasa de aprendizaje, número de neuronas, parámetro de regularización, etc) ¿El mejor parámetro para la red con sigmoidal es distinto que para ReLU? ¿Porqué sucede? Además mida el error real del modelo sobre el conjunto de pruebas, compare y concluya.

In [25]:
neuron_numbers = [64,128,192,256,320]

In [26]:
recompute = False:
    
if recompute:
    # Create the folds
    for activation in ("sigmoid","relu"):
        print("Activation %s:"%activation)
        for folds in (5,10):
            print("\tFolds %d:"%folds)
            Xm = X_trai_scaled.values
            ym = y_trai
            for nneurons in neuron_numbers:
                kfold = KFold(n_splits=folds,shuffle=False)
                cvscores = []
                for train, val in kfold.split(X_trai_scaled):
                    initi = keras.initializers.RandomUniform(-1e-5,1e-5,seed=42)
                    model = create_model(activation=activ,initializer=initi,neurons=nneurons)
                    model.fit(Xm[train],ym[train], epochs=EPOCHS, verbose=0,
                        batch_size=BATCH_SIZE)
                    scores = model.evaluate(Xm[val], ym[val], verbose=0)
                    cvscores.append(scores)
                mse_cv = np.mean(cvscores)
                mse_test = np.mean(model.evaluate(X_test_scaled,y_test, verbose=0))
                print("\t\t%4d Neurons:  mse_cv: %10.6f  mse_test: %10.6f"%(
                    nneurons,mse_cv,mse_test))

Activation sigmoid:
	Folds 5:
		  64 Neurons:  mse_cv:   0.316276  mse_test:   0.226028
		 128 Neurons:  mse_cv:   0.325131  mse_test:   0.220594
		 192 Neurons:  mse_cv:   0.333575  mse_test:   0.176910
		 256 Neurons:  mse_cv:   0.329223  mse_test:   0.084800
		 320 Neurons:  mse_cv:   0.329049  mse_test:   0.183090
	Folds 10:
		  64 Neurons:  mse_cv:   0.339446  mse_test:   0.230289
		 128 Neurons:  mse_cv:   0.366717  mse_test:   0.074861
		 192 Neurons:  mse_cv:   0.542468  mse_test:   0.300941
		 256 Neurons:  mse_cv:   0.302596  mse_test:   0.254113
		 320 Neurons:  mse_cv:   0.408010  mse_test:   0.213780
Activation relu:
	Folds 5:
		  64 Neurons:  mse_cv:   0.338098  mse_test:   0.176293
		 128 Neurons:  mse_cv:   0.310144  mse_test:   0.221150
		 192 Neurons:  mse_cv:   0.314514  mse_test:   0.194890
		 256 Neurons:  mse_cv:   0.307469  mse_test:   0.233891
		 320 Neurons:  mse_cv:   0.316362  mse_test:   0.178952
	Folds 10:
		  64 Neurons:  mse_cv:   0.908201  mse_test:   2.

Se decidió utilizar el **número de neuronas** como hiper parámetro. Para el caso de activación **sigmoide** el menor error se obtuvo con redes de 256 neuronas (0.084800) y para **relu** con 192 neuronas (0.180801).

La gran variación entre el **error CV de entrenamiento** el modelo entrenado para 5 Folds y su análogo de 10 Folds hacen ver que hay una variación importante producto de factores aleatorios, pudiendo ser la **elección del fold** (si los conjuntos son pequeños la variación del error puede ser alta), la **inicialización aleatoria**.

Se puede ver que el error obtenido en la **cross validation** no es una buena representación del error obtenido en el conjunto de test, pudiendo deberse a las mismas razones anteriores. Sin embargo se puede ver que el error con **5 Folds** es mucho más estable, puesto que trata con conjuntos más grandes. Los resultados no son concluyentes.